In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import json
from pathlib import Path
from tqdm import tqdm
from scipy.stats import pearsonr
import random
import matplotlib.pyplot as plt
from dataclasses import dataclass
import math

import biojepa_ac_model as model

## BioJEPA Model load

In [2]:
torch.manual_seed(1337)
random.seed(1337)

In [3]:
def get_device():
    device = 'cpu'
    if torch.cuda.is_available():
        torch.cuda.manual_seed(1337)
        device = 'cuda'
    # elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    #     device = 'mps'
    print(f'using {device}')
    return device

DEVICE = get_device()

using cpu


In [4]:
batch_size = 32
n_embd = 8

In [5]:
data_dir = Path('/Users/djemec/data/jepa')
eval_dir = data_dir / 'tokenized' / 'eval'

mask_path = eval_dir / 'binary_pathway_mask.npy'
metadata_path = eval_dir / 'perturbation_map.json'
checkpoint_dir = data_dir / 'checkpoint'
gene_names_path = eval_dir / 'gene_names.json'

In [6]:
print('Loading Pathway Mask...')
binary_mask = np.load(mask_path)
n_genes, n_pathways = binary_mask.shape
print(f'Mask Loaded: {n_genes} Genes -> {n_pathways} Pathways')

with open(metadata_path, 'r') as f:
    pert_map = json.load(f)
id_to_pert = {v: k for k, v in pert_map.items()}
print(f'Loaded {len(id_to_pert.keys())} perturbations')

with open(data_dir / 'gene_names.json', "r") as f:
    gene_names = json.load(f)
print(f'Loaded {len(gene_names)} genes')

Loading Pathway Mask...
Mask Loaded: 4096 Genes -> 1024 Pathways
Loaded 2058 perturbations
Loaded 4096 genes


In [7]:
config = model.BioJepaConfig(
    mask_matrix=binary_mask, 
    num_genes=n_genes,
    num_pathways=n_pathways,
    embed_dim=n_embd,
    heads=1
)
model = model.BioJepa(config).to(DEVICE)

**Load Checkpoint**

In [8]:
checkpoint_path = checkpoint_dir / 'bio_jepa_ckpt_4epoch_final.pt'
checkpoint = torch.load(checkpoint_path, map_location=DEVICE)

keys = model.load_state_dict(checkpoint['model'])
keys

<All keys matched successfully>

**Freeze Model**

In [9]:
model.eval()
for param in model.parameters():
    param.requires_grad = False

## Build Decoder

In [10]:
@dataclass
class BenchmarkDecoderConfig:
    embed_dim: int = 384
    num_pathways: int = 1024
    num_genes: int = 4096
    
class BenchmarkDecoder(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        # Step 1: Collapse the embedding dimension (384 -> 1)
        # This asks: "How active is this pathway overall?"
        self.pool = nn.Linear(config.embed_dim, 1) 
        
        # Step 2: Decode Pathway Activity -> Gene Expression
        # This learns the specific contribution of each pathway to each gene
        self.decode = nn.Linear(config.num_pathways, config.num_genes) 
        
    def forward(self, latents):
        # latents: [Batch, 1024, 384]
        
        # 1. Calculate Pathway Scores
        # [B, 1024, 384] -> [B, 1024, 1] -> [B, 1024]
        scores = self.pool(latents).squeeze(-1)
        
        # 2. Project to Genes
        # [B, 1024] @ [1024, 2000] -> [B, 2000]
        gene_preds = self.decode(scores)
        
        return gene_preds

### Data Loader

In [11]:
def load_shard(filename):
    print(f'loading {filename}') # Optional: reduce noise
    with np.load(filename) as data:
        # Load all arrays into memory
        # We convert to correct types immediately to save hassle later
        control_x = data['control'].astype(np.float32)
        control_tot = data['control_total'].astype(np.float32)
        case_x = data['case'].astype(np.float32)
        case_tot = data['case_total'].astype(np.float32)
        action_ids = data['action_ids'].astype(np.int64)
        
    return control_x, control_tot, case_x, case_tot, action_ids

class DataLoaderLite:
    def __init__(self, batch, split, device, tok_dir):
        self.batch = batch
        self.split = split
        self.device = device
        
        # 1. Find Shards
        data_root = tok_dir / f'{split}'
        shards = list(data_root.glob('*.npz'))

        self.total_files = len(shards)
        self.shards = sorted(shards)

        assert len(shards) > 0, f'no shards found for split {split}'
        print(f'found {len(shards)} shards for split {split}')
        
        self.reset()

    def reset(self):
        # Create a randomized queue of shards
        self.remaining_shards = list(self.shards)
        random.shuffle(self.remaining_shards)
        
        self.current_shard_idx = -1
        self.load_next_shard()

    def load_next_shard(self):
        self.current_shard_idx += 1
        
        # If we ran out of shards, reset (Epoch done)
        if self.current_shard_idx >= len(self.remaining_shards):
            self.reset() # This resets shard_idx to -1 and reshuffles
            return 

        # Load the file
        filename = self.remaining_shards[self.current_shard_idx]
        self.data_tuple = load_shard(filename)
        
        # Shuffle the items INSIDE the shard
        # This is critical so we don't just memorize the sorted order of the shard
        n_samples = len(self.data_tuple[0])
        self.perm = np.random.permutation(n_samples)
        self.current_position = 0
        self.total_samples_in_shard = n_samples

    def next_batch(self):
        batch = self.batch
        
        # Check if we have enough data left in current shard
        if self.current_position + batch > self.total_samples_in_shard:
            self.load_next_shard()
            # Recursively call to get batch from the new shard
            return self.next_batch()
            
        # Get indices for this batch
        indices = self.perm[self.current_position : self.current_position + batch]
        self.current_position += batch
        
        # Slice data using the shuffled indices
        # data_tuple structure: (xc, xct, xt, xtt, aid)
        batch_cont_x  = torch.from_numpy(self.data_tuple[0][indices]).to(self.device)
        batch_cont_tot = torch.from_numpy(self.data_tuple[1][indices]).to(self.device)
        batch_case_x  = torch.from_numpy(self.data_tuple[2][indices]).to(self.device)
        batch_case_t = torch.from_numpy(self.data_tuple[3][indices]).to(self.device)
        batch_aid = torch.from_numpy(self.data_tuple[4][indices]).to(self.device)
        
        return batch_cont_x, batch_cont_tot, batch_case_x, batch_case_t, batch_aid

**Data Loader**

In [12]:
train_loader = DataLoaderLite(batch=batch_size, split='train', device=DEVICE, tok_dir=eval_dir)
val_loader = DataLoaderLite(batch=batch_size, split='val', device=DEVICE, tok_dir=eval_dir)


found 11 shards for split train
loading /Users/djemec/data/jepa/tokenized/eval/train/shard_train_0000.npz
found 2 shards for split val
loading /Users/djemec/data/jepa/tokenized/eval/val/shard_val_0001.npz


## Training Decoder

### Training Config/Setup

In [13]:
lr_decoder = 4e-3
epochs = 10
tok_file_chunk_size = 10000

**Initialize Decoder** 

In [14]:
config = BenchmarkDecoderConfig(
    embed_dim= n_embd,
    num_pathways= n_pathways,
    num_genes= n_genes
)

decoder = BenchmarkDecoder(config).to(DEVICE)

**Optimizer**

In [15]:
optimizer = torch.optim.AdamW(decoder.parameters(), lr=lr_decoder)

**Training Lenght**

In [16]:
train_total_examples = 101682
val_total_examples = 11044
test_total_examples = 38829

In [17]:
steps_per_epoch = train_total_examples // batch_size
max_steps = epochs * steps_per_epoch
steps_per_epoch, max_steps

(3177, 31770)

**Scheduler**

In [18]:
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer, max_lr=lr_decoder, total_steps=max_steps, pct_start=0.05
)

### Training Loop

In [19]:
lossi = []
step = 0
total_epoch_loss = 0

In [ ]:
for step in range(max_steps):

    last_step = (step == max_steps - 1)

    # once in a while evaluate our validation set loss
    if step % 100 == 0 or last_step:
        model.eval()
        with torch.no_grad():
            val_loss_accum = 0.0
            val_loss_steps = 25
            for i in range(val_loss_steps):
                cont_x, cont_tot, case_x, case_tot, act_id = val_loader.next_batch()

                # run BioJEPA
                with torch.no_grad():
                    z_context = model.student(cont_x, cont_tot)
                    z_pred = model.predictor(z_context, act_id)

                # run new decoder
                pred_delta = decoder(z_pred) - decoder(z_context)
                real_delta = case_x - cont_x

                loss = F.mse_loss(pred_delta, real_delta)
                loss = loss / val_loss_steps
                val_loss_accum += loss.detach()


                

        print(f'test loss: {val_loss_accum.item():.4f}')


    # periodically save checkpoint
    if step > 0 and (step % 1000 == 0 or step % steps_per_epoch ==0) and not last_step:
        # Save Checkpoint
        torch.save({
            'model': decoder.state_dict(),
            'optimizer': optimizer.state_dict(),
            'step': step
        }, checkpoint_dir / f'biojepa_decoder_ckpt_{step}.pt')

    # actual training
    decoder.train
    cont_x, cont_tot, case_x, case_tot, act_id = train_loader.next_batch()

    # run frozen BioJEPA
    with torch.no_grad():
        z_context = model.student(cont_x, cont_tot)
        z_pred = model.predictor(z_context, act_id)

    # run decoder
    pred_delta = decoder(z_pred) - decoder(z_context)
    real_delta = case_x - cont_x
    real_delta = case_x - cont_x

    # loss
    pred_delta = decoder(z_pred) - decoder(z_context)
    real_delta = case_x - cont_x

    optimizer.zero_grad()

    loss = F.mse_loss(pred_delta, real_delta)
    loss.backward()

    optimizer.step()
    scheduler.step()

    # loss caching
    lossi.append(loss.item())
    total_epoch_loss += loss.item()

    if step % 25 == 0:
        print(f"Step {step} | Loss: {loss.item():.5f} | LR: {scheduler.get_last_lr()[0]:.2e}")
    
    
    if step > 0 and step % steps_per_epoch == 0:   
        avg_loss = total_epoch_loss / steps_per_epoch
        print(f"=== Step {step} Done. Avg Loss: {avg_loss:.5f} ===")
        total_epoch_loss = 0
    
    if last_step:
        # Save Checkpoint
        torch.save({
            'model': decoder.state_dict(),
            'optimizer': optimizer.state_dict(),
            'step': step
        }, checkpoint_dir / f'biojepa_decoder_ckpt_{step}_final.pt')

    step += 1

test loss: 0.8973
Step 0 | Loss: 0.91202 | LR: 1.60e-04
Step 25 | Loss: 0.86784 | LR: 1.63e-04
Step 50 | Loss: 0.89279 | LR: 1.70e-04
Step 75 | Loss: 0.87640 | LR: 1.82e-04
loading /Users/djemec/data/jepa/tokenized/eval/val/shard_val_0000.npz
test loss: 0.8991
Step 100 | Loss: 0.92670 | LR: 1.98e-04
Step 125 | Loss: 0.88284 | LR: 2.19e-04
Step 150 | Loss: 0.89942 | LR: 2.45e-04


**Training Loss Plot**

In [ ]:
plt.plot(lossi)
plt.yscale('log')

## Trained Decoder Validation

In [ ]:
decoder.eval()
correlations = []
mses = []

In [ ]:
val_steps_per_epoch = val_total_examples // batch_size
test_steps_per_epoch = test_total_examples // batch_size

In [ ]:
for step in tqdm(range(val_steps_per_epoch), desc="Evaluating"):
    
    # Custom Loader Call
    cont_x, cont_tot, case_x, case_tot, act_id = val_loader.next_batch()
    
    with torch.no_grad():
        z_context = model.student(cont_x, cont_tot)
        z_pred = model.predictor(z_context, act_id)
        
        pred_delta = decoder(z_pred) - decoder(z_context)
        real_delta = case_x - cont_x
        
    # Per-Sample Metrics
    for i in range(len(pred_delta)):
        p = pred_delta[i].cpu().numpy()
        t = real_delta[i].cpu().numpy()
        
        # Metric: Pearson Correlation on Top 20 DEGs
        top_20_idx = np.argsort(np.abs(t))[-20:]
        
        if np.std(p[top_20_idx]) > 1e-9 and np.std(t[top_20_idx]) > 1e-9:
            corr, _ = pearsonr(p[top_20_idx], t[top_20_idx])
            correlations.append(corr)
            
        mses.append(np.mean((p - t)**2))

In [ ]:
mean_mse = np.mean(mses)
mean_corr = np.mean(correlations)
print(f'Global MSE: {mean_mse:.4f}')
print(f'Top-20 Pearson R: {mean_corr:.4f}')

In [ ]:
if mean_corr > 0.75:
    print('🌟 SOTA COMPETITIVE (Matches GEARS)')
elif mean_corr > 0.40:
    print('✅ FUNCTIONAL (Better than random)')
else:
    print('❌ NEEDS IMPROVEMENT')
